# MIE1624H - Watson Analytics - Tone Analyzer API Tutorial

The Watson Analytics Tone Analyzer service analyzes the provided text or document for a variety of tones at both the document, and sentence level. It detects three types of tones: Emotion (anger, disgust, fear, joy, and sadness), social propensities (openness, conscientiousness, extroversion, agreeableness, and emotional range), and language styles (analytical, confident, and tentative). 

More information regarding this service can be found in the IBM Watson documentation:
https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/


#### Download necessary libraries

In [1]:
!pip install watson-developer-cloud

#### Import necessary libraries

In [2]:
from watson_developer_cloud import ToneAnalyzerV3 as TA
import json

### Insert API credentials from Bluemix
This is found by going the API's page and looking under 'Service Credentials'. The username and password can be viewed by clicking 'View Credentials' next to your Key Name.

Copy the credentials and replace the contents of *watson_credentials.json*. Alternatively, you can just copy and paste the keys in the variables for username and password.

#### Parse watson_credentials.json file

In [3]:
credentials = {}

with open('watson_credentials.json') as f:
    data = json.load(f)
    for k in data.keys():
        try:
            credentials[k] = {
                'username': data[k][0]['credentials']['username'],
                'password': data[k][0]['credentials']['password']
            }
        except KeyError:
            credentials[k] = {
                'api_key': data[k][0]['credentials']['api_key']
            }

#### Get username and password for TA
Enter API username and password manually or add to watson credentials file.

In [4]:
username = ''
password = ''

if username == '' and password == '':
    username = credentials['tone_analyzer']['username']
    password = credentials['tone_analyzer']['password']

## Create Natural Language Classifier Instance

In [5]:
ta = TA(
    username=username,
    password=password,
    version='2017-09-21'
)

## Analyze General Tone

The general tone endpoint can be used to analyze the input for several types of tones (emotion, language, and social).

Arguments:
    - text (str): The plain text input to be analyzed (max: 128 kB)
    - tones (str,[str,..]): comma separated list of types of tones to return. By default all are active. Options: ['emotion','language',social']
    - sentences (bool): Whether or not to analyze tone by sentence or not. Default = true
    - content type (str): 'text/plain' for plain text (default), 'text/html' for html formatted text, and 'application/json' for json formatted text 
    e.g. tone(text, tones='emotion')

Only the text argument is required in the above arguments. 

## General Function to Analyze Tone

In [6]:
def getTones(txt, categories='', sentences=True):
    if categories == '':
        response = ta.tone({'text': txt},sentences=sentences)
    else:
        response = ta.tone({'text': txt},tones=categories,sentences=sentences)
    
    print ('Raw Output')
    print (json.dumps(response,indent=2))
    
    # Populate dictionary
    tones = {}
    
    for i in range(len(response['document_tone']['tones'])):
        tone_name = response['document_tone']['tones'][i]['tone_name']
        tones[tone_name] = response['document_tone']['tones'][i]['score']

    return tones

## Analyzing a String

Here the tone analyzer API is called with the below tweet and the raw output is displayed.

In [7]:
sample1 = "can't decide if i should even watch the #democraticdebate or is it not worth the migraine!? #conservative #republican2016 #tcot #ycot"

In [8]:
tones = getTones(sample1)
print ('\nDictionary Format')
print (tones)

Raw Output
{
  "document_tone": {
    "tones": [
      {
        "tone_id": "analytical",
        "score": 0.882284,
        "tone_name": "Analytical"
      },
      {
        "tone_id": "tentative",
        "score": 0.905166,
        "tone_name": "Tentative"
      }
    ]
  }
}

Dictionary Format
{'Analytical': 0.882284, 'Tentative': 0.905166}


## Analyzing a Text Document

In [9]:
with open('tone.json') as tone_json:
    response = ta.tone(tone_json.read())

print(json.dumps(response, indent=2))

{
  "document_tone": {
    "tones": [
      {
        "tone_id": "sadness",
        "score": 0.6165,
        "tone_name": "Sadness"
      },
      {
        "tone_id": "analytical",
        "score": 0.829888,
        "tone_name": "Analytical"
      }
    ]
  },
  "sentences_tone": [
    {
      "text": "Team, I know that times are tough!",
      "tones": [
        {
          "tone_id": "analytical",
          "score": 0.801827,
          "tone_name": "Analytical"
        }
      ],
      "sentence_id": 0
    },
    {
      "text": "Product sales have been disappointing for the past three quarters.",
      "tones": [
        {
          "tone_id": "sadness",
          "score": 0.771241,
          "tone_name": "Sadness"
        },
        {
          "tone_id": "analytical",
          "score": 0.687768,
          "tone_name": "Analytical"
        }
      ],
      "sentence_id": 1
    },
    {
      "text": "We have a competitive product, but we need to do a better job of selling it!",
 

In [10]:
print("The string: \n{}".format(sample1))
tones = {}

for i in range(len(response['document_tone']['tones'])):
    tone_name = response['document_tone']['tones'][i]['tone_name']
    tones[tone_name] = response['document_tone']['tones'][i]['score']

print ("\nThe document tones:", tones)

The string: 
can't decide if i should even watch the #democraticdebate or is it not worth the migraine!? #conservative #republican2016 #tcot #ycot

The document tones: {'Analytical': 0.829888, 'Sadness': 0.6165}


## Analyzing a JSON Conversation

Here a JSON conversation is analyzed.

First here is the raw JSON output from the analyzer.

In [11]:
with open('tone-chat.json') as tone_json:
  response = ta.tone_chat(json.load(tone_json)['utterances'])

print ('Raw Output')
print(json.dumps(response, indent=2))

Raw Output
{
  "utterances_tone": [
    {
      "utterance_text": "Hello, I'm having a problem with your product.",
      "utterance_id": 0,
      "tones": [
        {
          "tone_id": "polite",
          "score": 0.718352,
          "tone_name": "Polite"
        }
      ]
    },
    {
      "utterance_text": "OK, let me know what's going on, please.",
      "utterance_id": 1,
      "tones": []
    },
    {
      "utterance_text": "Well, nothing is working :(",
      "utterance_id": 2,
      "tones": [
        {
          "tone_id": "sad",
          "score": 0.997149,
          "tone_name": "sad"
        }
      ]
    },
    {
      "utterance_text": "Sorry to hear that.",
      "utterance_id": 3,
      "tones": [
        {
          "tone_id": "polite",
          "score": 0.689109,
          "tone_name": "Polite"
        },
        {
          "tone_id": "sympathetic",
          "score": 0.663203,
          "tone_name": "Sympathetic"
        }
      ]
    }
  ]
}


Now lets intrepret the output. Below, the user and what they said and the corresponding tones associated with what was said is shown.

In [12]:
names = []
with open('tone-chat.json') as tone_json:
    utterances = json.load(tone_json)['utterances']
    for i in range(len(utterances)):
        names.append(utterances[i]['user'])


for i in range(len(response['utterances_tone'])):
    tones = []
    for tone in response['utterances_tone'][i]['tones']:
        tones.append(tone['tone_name'])
    print (names[i],":",response['utterances_tone'][i]['utterance_text'],tones)

customer : Hello, I'm having a problem with your product. ['Polite']
agent : OK, let me know what's going on, please. []
customer : Well, nothing is working :( ['sad']
agent : Sorry to hear that. ['Polite', 'Sympathetic']
